In [1]:
import numpy as np
import pandas as pd
import csv
import os
import nltk
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem 

In [2]:
path_dev_news = '/home/tingchih/ml/Project/dev/dev_news.csv'
path_dev_behaviors = '/home/tingchih/ml/Project/dev/behaviors.tsv'

In [3]:
beha_column = ['ImpressionID','userID','time','history','impressions']
dev_beha = pd.read_table(path_dev_behaviors,header=None,names=beha_column)

In [30]:
"""
1. example: Dictionary news:{'news title1':1 , 'news title2':0 , ....}
2. split news Dictionary to training set, validation set, testing set
3. training set(x_train,y_train) , validation set(x_val,y_val) , testing set(x_test,y_test)
"""

"\n1. example: Dictionary news:{'news title1':1 , 'news title2':0 , ....}\n2. split news Dictionary to training set, validation set, testing set\n3. training set(x_train,y_train) , validation set(x_val,y_val) , testing set(x_test,y_test)\n"

In [ ]:
# from behaviors.tsv extract news ID and impression
#imp = dev_beha['impressions'][8]
#a = split_beha_info(imp)
#for key in sorted(a.keys()):
#    print(key,a[key])

newsID , impression = make_news_dictionary()

In [13]:
print(type(impression[0]))
count = 0
for i in range(0,len(impression)):
    if impression[i] == 1:
        count = count+1
print(count)

<class 'int'>
192


In [ ]:
news_info = []
for i in range(0,len(newsID)):
    text = extract_news(newsID[i])
    news_info.append(text)
    print(i)

In [15]:
print(len(news_info))
print(len(impression))

4780
4780


In [16]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

In [17]:
for i in range(0,2868):
    x_train.append(news_info[i])
    y_train.append(impression[i])
for i in range(2868,3824):
    x_val.append(news_info[i])
    y_val.append(impression[i])
for i in range(3824,4780):
    x_test.append(news_info[i])
    y_test.append(impression[i])

In [18]:
print(len(x_train),len(y_train))
print(len(x_val),len(y_val))
print(len(x_test),len(y_test))

2868 2868
956 956
956 956


In [28]:
print(x_train[0])
print(type(x_train[0]))

The most talked about movie moments of the 2010sLet's try to emphasize the positive, but there's one particular sequence that was so special in its awfulness, it had to go on this list. See if you can guess which one it is.
<class 'str'>


In [32]:
text_file = open("x_train.txt", "w")
for i in range(len(x_train)):
    text_file.write(x_train[i])
text_file.close()

In [6]:
def make_news_dictionary():
    #news = {}
    nID = []
    record = []
    for k in range(0,42417):
        impresson = dev_beha['impressions'][k] # impression is a string ex: N12334-1 N4566-0 ...
        individual_impression = split_beha_info(impresson)
        for key in sorted(individual_impression.keys()):
            if key not in nID:
                nID.append(key)
                record.append(individual_impression[key])
                print(key,individual_impression[key])

    return nID , record

In [7]:
def extract_news(newsID):
    
    with open(path_dev_news, newline='',encoding='utf8') as csvfiler:
        rows = csv.DictReader(csvfiler)
        for row in rows:
            if row['NewsID'] == newsID:
                title = row['Title']
                abstract = row['Abstract']
                total = title + abstract
    return total

In [8]:
def extract_behaviors(uID):
    impression = []
    for i in range(0,len(dev_beha)):
        if dev_beha['userID'][i] == uID:
            impression.append(dev_beha['impressions'][i])
            
    return impression

In [9]:
def split_news_info(Array):
    if len(Array) == 0:
        return
    else:
        ans = []
        count = ''
        length = len(Array)
        for i in range(0,length):
            for j in range(0,len(Array[i])):
                if Array[i][j] == 'N':
                    count += Array[i][j]
                elif Array[i][j] == ' ':
                    ans.append(count)
                    count = ''
                elif j == len(Array[i])-1:
                    count += Array[i][j]
                    ans.append(count)
                    count = ''
                else:
                    #0-9
                    count += Array[i][j]
        return ans

In [10]:
def split_beha_info(impress):# impress is string
    ans = {}
    # example: ans ={'N55189':1,'N42782':1,...}
    newsID = ''
    length = len(impress)
    for i in range(0,length):
        if impress[i] == 'N':
            newsID = ''
            newsID += impress[i]
        elif impress[i] == '-':
            ans[newsID] = int(impress[i+1])
            newsID = ''
        else:
            #0-9
            newsID += impress[i]
    return ans

In [11]:
def extract_impression_news(dictionary):
    dictionary_news = []
    read = []
    for news,impression in dictionary.items():
        dictionary_news.append(news)
        read.append(impression)
    ans = []
    for i in range(0,len(dictionary_news)):
        with open(path_train_news, newline='',encoding='utf8') as csvfiler:
            rows = csv.DictReader(csvfiler)
            for row in rows:
                if row['NewsID'] == dictionary_news[i]:
                    title = row['Title']
                    ans.append(title)
            
    return ans , read